# Summarize News articles from main page of a news website
## Spacy Library
Decided to switch to Spacy which is better instead of flair

## Main Ideas and How the results can be used
* Sift through news sites  
* filter news that mentions name of brand  
* summarize article to get gist of news
* do a timeline of sentiment using sentiment analysis
* Regression of sentiment on any other factor
* STOCK price vs Sentiment in news
* number of sales and sentiment in reviews 

### Sift through news articles using python library *newspaper3k*

In [ ]:
# pip install newspaper3k

In [1]:
import newspaper
from newspaper import Article
import re

In [2]:
# build newspaper object 
wsj_paper = newspaper.build('https://www.cnn.com/', memoize_articles=False)
print(len(wsj_paper.articles)) # number of articles on the fron page

716


In [3]:
# get urls of all the articles and store in a list
urls = []
for article in wsj_paper.articles:
    urls.append(article.url)

In [4]:
# texts contains all articles in CNN front page that mention Apple
texts  = []
for u in urls[:200]:
    article = Article(u)
    article.download()
    article.parse()
    if "Apple" in article.text:
        texts.append(article.text)
    

In [ ]:
# this command in its own cell, otherwise won't work
#pip install -U spacy 


In [5]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
# make a list of stopwords, import punctuation and add newline to it

stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

need to do below command for a better trained dictionary en_core_web_sm

In [ ]:
# !python3 -m spacy download en_core_web_sm

In [6]:
nlp = spacy.load('en_core_web_sm')

for finding score of each word in a sentence, we use pytextrank


In [ ]:
#pip install pytextrank

In [10]:
import pytextrank

In [11]:
## add PyTextRank to the spaCy pipeline (do it once only otherwise it gives error)
nlp.add_pipe("textrank")


In [12]:
nlp.pipe_names # check if textrank got added

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textrank']

In [28]:
# define function for cleaning sentences before applying textrank that will help later when calculating score of sentences.

def clean_text(t):
    return re.sub("[^a-zA-Z\.\?\!]", " ",t)

In [51]:
# function that takes in set of sentences or a text block(t) and gives dictionay of score of keyphrases
def dict_kphrase(t):
    dict_kp = dict() 
    doc = nlp(clean_text)
    for phrase in doc._.phrases:
        dict_kp[phrase.text]=phrase.rank
    return dict_kp
        #myList.append((phrase.rank, phrase.text))

In [15]:
# for spliting text into sentences we will use nltk
import nltk
nltk.download('punkt') # one time execution


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nadiakhalil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
sentences = nltk.sent_tokenize(clean_text)

In [45]:
# function that gives dictionary for calculating score of each sentence using keyphrase scores. Takes in t=text of
# sentences (sentences) and a dictionary (d) of scores of keyphrases of that text (t)
def score(t,d):
    score_dict = dict()
    for sent in t:
        score_dict[sent] = 0
        for kphrase in d.keys():
            if kphrase in sent:
                score_dict[sent]+= d[kphrase][0]
    return score_dict
    

### Summary
The idea is to consider n highest ranked sentences as summary of the article.

In [46]:
from heapq import nlargest

In [47]:
ThreeHighest = nlargest(3, score_dict, key = score_dict.get)

In [48]:
print(ThreeHighest) 

[' CNN  The cast and producers of Apple s  Ted Lasso  scored a major W during Tuesday s Emmy nominations and broke a record in the process.', 'With its    nominations   Ted Lasso  became the most nominated freshman comedy in Emmy history  besting the    nominations  Glee  earned in      for its first season.', 'The nods earned by  Ted Lasso  included best comedy  best lead actor  Jason Sudeikis  and a number of accolades for the show s supporting cast.']
